In [1]:
import tensorflow as tf
import numpy as np
from pprint import pprint
print(tf.__version__)

1.12.0


In [2]:
sentences = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]

In [3]:
# word dic
word_list = []
for elm in sentences:
    word_list += elm
word_list = list(set(word_list))
word_list.sort()
word_list = ['<pad>'] + word_list # '<pad>'라는 의미없는 token 추가
word_dic = {word : idx for idx, word in enumerate(word_list)}
pprint(word_dic)

{'<pad>': 0,
 'I': 1,
 'a': 2,
 'changing': 3,
 'deep': 4,
 'difficult': 5,
 'fast': 6,
 'feel': 7,
 'for': 8,
 'framework': 9,
 'hungry': 10,
 'is': 11,
 'learning': 12,
 'tensorflow': 13,
 'very': 14}


In [4]:
# max_len의 길이에 못미치는 문장은 <pad>로 max_len만큼 padding
def pad_seq(sequences, max_len, dic):
    seq_len, seq_indices = [], []
    for seq in sequences:
        seq_len.append(len(seq))
        seq_idx = [dic.get(char) for char in seq]
        seq_idx += (max_len - len(seq_idx)) * [dic.get('<pad>')] # 0 is idx of meaningless token "<pad>"
        seq_indices.append(seq_idx)
    return seq_len, seq_indices

In [5]:
max_length = 8
sen_len, sen_indices = pad_seq(sequences = sentences, max_len = max_length, dic = word_dic)
pprint(sen_len)
pprint(sen_indices)

[3, 4, 7, 5]
[[1, 7, 10, 0, 0, 0, 0, 0],
 [13, 11, 14, 5, 0, 0, 0, 0],
 [13, 11, 2, 9, 8, 4, 12, 0],
 [13, 11, 14, 6, 3, 0, 0, 0]]


## Intro: Padding

In [6]:
seq_len = tf.placeholder(dtype = tf.int32, shape = [None])
seq_indices = tf.placeholder(dtype = tf.int32, shape = [None, max_length])

In [7]:
one_hot = np.eye(len(word_dic)).astype(np.float32)
one_hot = tf.get_variable(name='one_hot', initializer = one_hot,
                                   trainable = False) # embedding vector training 안할 것이기 때문에~
seq_batch = tf.nn.embedding_lookup(params = one_hot, ids = seq_indices)

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tmp = sess.run(seq_batch, feed_dict = {seq_indices : sen_indices})
print(np.shape(sen_indices))
print(np.shape(tmp)) # tf.nn.dynamic_rnn, tf.contrib.seq2seq.TrainingHelper 등에 이 shape을 유지하면서 전달되어야함

(4, 8)
(4, 8, 15)


In [9]:
tmp[0]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.]], dtype=float32)

## Many to One

In [28]:
tf.reset_default_graph()

sentences = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]

# label이 갖는 의미가 뭘까?
# tensorflow에 대해서 긍정적인 구문을 갖는 것을 Classification하겠다.
# Sentimental
y = [[0.,1.], [0.,1.], [1.,0.], [1.,0.]]
max_length = 8

sen_len, sen_indices = pad_seq(sequences = sentences, max_len = max_length, dic = word_dic)

pprint(sen_len)
pprint(sen_indices)

change_to_np = np.asarray(sen_indices)
print(change_to_np.shape)

[3, 4, 7, 5]
[[1, 7, 10, 0, 0, 0, 0, 0],
 [13, 11, 14, 5, 0, 0, 0, 0],
 [13, 11, 2, 9, 8, 4, 12, 0],
 [13, 11, 14, 6, 3, 0, 0, 0]]
(4, 8)


In [29]:
max_length = 8

# 모델 하이퍼 파라미터
h_dim = 2
n_of_classes = 2

seq_len = tf.placeholder(dtype = tf.int32, shape = [None])
seq_indices = tf.placeholder(dtype = tf.int32, shape = [None, max_length])
label = tf.placeholder(dtype = tf.float32, shape = [None, 2])

one_hot = np.eye(len(word_dic)).astype(np.float32)
one_hot = tf.get_variable(name='one_hot', initializer = one_hot,
                                   trainable = False)
seq_batch = tf.nn.embedding_lookup(params = one_hot, ids = seq_indices)
print(seq_batch)

Tensor("embedding_lookup/Identity:0", shape=(?, 8, 15), dtype=float32)


In [35]:
gru_cell = tf.contrib.rnn.GRUCell(num_units = h_dim, activation = tf.nn.tanh)

# return value는 output, state
# length를 넣는 이유는 rnn 크기를 동적으로 변경하기 위해서?
_, state = tf.nn.dynamic_rnn(cell = gru_cell, inputs = seq_batch, sequence_length = seq_len,
                             dtype = tf.float32)

# output의 경우 각 elements에 대해서 2 dimension이 추가로 붙어있는거 같은데, 이게 의미하는게 뭘까?
# 왜 하나의 elements 마다 output값이 존재할까?
# 원래 각 cell에 대해서는 output이 존재함
# state값은 뭘까?
# state값은 h값
pprint(_)
pprint(state)

ValueError: Variable rnn/gru_cell/gates/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-30-255c78fab7ce>", line 5, in <module>
    dtype = tf.float32)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3185, in run_ast_nodes
    if (yield from self.run_code(code, result)):


In [31]:
score = tf.layers.dense(inputs = state, units = n_of_classes)
ce_loss = tf.losses.softmax_cross_entropy(onehot_labels=label,
                                          logits = score)

In [33]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
result = sess.run([_, state], feed_dict = {seq_len : [sen_len[0]], seq_indices : [sen_indices[0]]})
pprint(result[0])
pprint(result[0].shape)
pprint(result[1])
pprint(result[1].shape)

array([[[-0.10585326,  0.12104207],
        [-0.07837567,  0.08045947],
        [-0.14166801,  0.01289127],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32)
(1, 8, 2)
array([[-0.14166801,  0.01289127]], dtype=float32)
(1, 2)


In [34]:
pprint(sess.run(ce_loss, feed_dict = {seq_len : sen_len, seq_indices : sen_indices,
                                      label : y}))
sess.close()

0.72538799


## Many to One : Stacked

In [40]:
tf.reset_default_graph()

In [41]:
max_length = 8
n_of_classes = 2

seq_len = tf.placeholder(dtype = tf.int32, shape = [None])
seq_indices = tf.placeholder(dtype = tf.int32, shape = [None, max_length])
label = tf.placeholder(dtype = tf.float32, shape = [None, 2])
keep_prob = tf.placeholder(dtype = tf.float32)

one_hot = np.eye(len(word_dic)).astype(np.float32)
one_hot = tf.get_variable(name='one_hot', initializer = one_hot,
                                   trainable = False)
seq_batch = tf.nn.embedding_lookup(params = one_hot, ids = seq_indices)

In [42]:
# DropoutWrapper 무엇.....
# MultiRNNCell 무엇....
# cell에 열심히 이어붙이자!
h_dims = [2,2]
gru_cells = []
for h_dim in h_dims:
    gru_cell = tf.contrib.rnn.GRUCell(num_units = h_dim, activation = tf.nn.tanh)
    gru_cell = tf.contrib.rnn.DropoutWrapper(cell = gru_cell,
                                             output_keep_prob = keep_prob)
    gru_cells.append(gru_cell)
else:
    # 완료되면 object를 가지고 있는 list를 인자로 받는 묶어주는 api만드는 느낌인건가 그런건가?
    gru_cells = tf.contrib.rnn.MultiRNNCell(cells = gru_cells)

In [43]:
_, state = tf.nn.dynamic_rnn(cell = gru_cells, inputs = seq_batch, sequence_length = seq_len,
                             dtype = tf.float32)
pprint(_)
pprint(state)

<tf.Tensor 'rnn/transpose_1:0' shape=(?, 8, 2) dtype=float32>
(<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 2) dtype=float32>)


In [44]:
score = tf.layers.dense(inputs = state[-1], units = n_of_classes)
ce_loss = tf.losses.softmax_cross_entropy(onehot_labels = label, logits = score)

In [45]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
pprint(sess.run([_, state], feed_dict = {seq_len : [sen_len[0]], seq_indices : [sen_indices[0]],
                                         keep_prob : 1.}))

[array([[[ 0.01601961,  0.00048447],
        [ 0.03477339,  0.02341287],
        [ 0.00379085,  0.03036179],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32),
 (array([[ 0.08877894,  0.05552789]], dtype=float32),
  array([[ 0.00379085,  0.03036179]], dtype=float32))]


In [46]:
pprint(sess.run(ce_loss, feed_dict = {seq_len : sen_len, seq_indices : sen_indices,
                                      label : y, keep_prob : 1.}))

0.67769277


In [47]:
sess.close()

## Many to One : Bi-directional

In [48]:
tf.reset_default_graph()

In [49]:
# Bi-directional RNN은 왜쓰나영?
max_length = 8
h_dim = 2
n_of_classes = 2

seq_len = tf.placeholder(dtype = tf.int32, shape = [None])
seq_indices = tf.placeholder(dtype = tf.int32, shape = [None, max_length])
label = tf.placeholder(dtype = tf.float32, shape = [None, 2])

one_hot = np.eye(len(word_dic)).astype(np.float32)
one_hot = tf.get_variable(name='one_hot', initializer = one_hot,
                                   trainable = False)
seq_batch = tf.nn.embedding_lookup(params = one_hot, ids = seq_indices)

In [50]:
gru_fw_cell = tf.contrib.rnn.GRUCell(num_units = h_dim, activation = tf.nn.tanh)
gru_bw_cell = tf.contrib.rnn.GRUCell(num_units = h_dim, activation = tf.nn.tanh)

_, output_states = tf.nn.bidirectional_dynamic_rnn(cell_fw = gru_fw_cell, cell_bw = gru_bw_cell,
                                           inputs = seq_batch, sequence_length = seq_len, dtype = tf.float32)
pprint(_)
pprint(output_states)

(<tf.Tensor 'bidirectional_rnn/fw/fw/transpose_1:0' shape=(?, 8, 2) dtype=float32>,
 <tf.Tensor 'ReverseSequence:0' shape=(?, 8, 2) dtype=float32>)
(<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 2) dtype=float32>)


In [51]:
# fw_cell의 final state와 bw_cell의 final state를 concatenate
concat_state = tf.concat(values = [output_states[0],
                                   output_states[-1]],
                         axis = 1)

In [52]:
score = tf.layers.dense(inputs = concat_state, units = n_of_classes)
ce_loss = tf.losses.softmax_cross_entropy(onehot_labels = label, logits = score)

In [53]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
pprint(sess.run([_, concat_state], feed_dict = {seq_len : [sen_len[0]], seq_indices : [sen_indices[0]]}))

[(array([[[-0.08163253,  0.07380992],
        [-0.03196622,  0.19157514],
        [ 0.04774579,  0.22382909],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32),
  array([[[-0.04967874, -0.03964158],
        [ 0.11081116,  0.09423791],
        [ 0.05272381,  0.07070614],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32)),
 array([[ 0.04774579,  0.22382909, -0.04967874, -0.03964158]], dtype=float32)]


In [54]:
pprint(sess.run(ce_loss, feed_dict = {seq_len : sen_len, seq_indices : sen_indices,
                                      label : y}))

0.64690757


## Many to One : Stacked Bi-directional

In [55]:
tf.reset_default_graph()

In [56]:
max_length = 8
n_of_classes = 2

seq_len = tf.placeholder(dtype = tf.int32, shape = [None])
seq_indices = tf.placeholder(dtype = tf.int32, shape = [None, max_length])
label = tf.placeholder(dtype = tf.float32, shape = [None, 2])
keep_prob = tf.placeholder(dtype = tf.float32)

one_hot = np.eye(len(word_dic)).astype(np.float32)
one_hot = tf.get_variable(name='one_hot', initializer = one_hot,
                                   trainable = False)
seq_batch = tf.nn.embedding_lookup(params = one_hot, ids = seq_indices)


In [57]:
h_dims = [2,2]
gru_fw_cells, gru_bw_cells = [], []

# forward
for h_dim in h_dims:
    gru_cell = tf.contrib.rnn.GRUCell(num_units = h_dim, activation = tf.nn.tanh)
    gru_cell = tf.contrib.rnn.DropoutWrapper(cell = gru_cell, output_keep_prob = keep_prob)
    gru_fw_cells.append(gru_cell)
    
# backward
for h_dim in h_dims:
    gru_cell = tf.contrib.rnn.GRUCell(num_units = h_dim, activation = tf.nn.tanh)
    gru_cell = tf.contrib.rnn.DropoutWrapper(cell = gru_cell, output_keep_prob = keep_prob)
    gru_bw_cells.append(gru_cell)

In [58]:
outputs, output_state_fw, output_state_bw = \
tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw = gru_fw_cells, cells_bw = gru_bw_cells,
                                               inputs = seq_batch, sequence_length = seq_len,
                                               dtype = tf.float32)
pprint(outputs)
pprint(output_state_fw)
pprint(output_state_bw)

<tf.Tensor 'stack_bidirectional_rnn/cell_1/concat:0' shape=(?, 8, 4) dtype=float32>
(<tf.Tensor 'stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'stack_bidirectional_rnn/cell_1/bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 2) dtype=float32>)
(<tf.Tensor 'stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 2) dtype=float32>)


In [59]:
# 왜 맨 마지막꺼만 가져가냐!!
# 아... 맨 마지막 state값이 중요해서?
concat_state = tf.concat(values=[output_state_fw[-1],output_state_bw[-1]], axis = 1)

In [60]:
score = tf.layers.dense(inputs = concat_state, units = n_of_classes)
ce_loss = tf.losses.softmax_cross_entropy(onehot_labels = label, logits = score)

In [61]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
pprint(sess.run([outputs, concat_state], feed_dict = {seq_len : [sen_len[0]], seq_indices : [sen_indices[0]],
                                                      keep_prob : 1.}))

[array([[[ 0.02087362,  0.02930984, -0.04963552, -0.01023802],
        [ 0.04402361,  0.02402181, -0.01032312,  0.00146264],
        [ 0.05441971, -0.03184908,  0.00424442,  0.00892211],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]], dtype=float32),
 array([[ 0.05441971, -0.03184908, -0.04963552, -0.01023802]], dtype=float32)]


In [62]:
pprint(sess.run(ce_loss, feed_dict = {seq_len : sen_len, seq_indices : sen_indices,
                                      label : y, keep_prob : 1.}))

0.68557513


## Many to Many

In [63]:
tf.reset_default_graph()

sentences = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
pos = [['pronoun', 'verb', 'adjective'],
     ['noun', 'verb', 'adverb', 'adjective'],
     ['noun', 'verb', 'determiner', 'noun', 'preposition', 'adjective', 'noun'],
     ['noun', 'verb', 'adverb', 'adjective', 'verb']]
max_length = 8

In [64]:
# max_len의 길이에 못미치는 문장은 <pad>로 max_len만큼 padding
def pad_seq(sequences, max_len, dic):
    seq_len, seq_indices = [], []
    for seq in sequences:
        seq_len.append(len(seq))
        seq_idx = [dic.get(char) for char in seq]
        seq_idx += (max_len - len(seq_idx)) * [dic.get('<pad>')] # 0 is idx of meaningless token "pad"
        seq_indices.append(seq_idx)
    return seq_len, seq_indices

In [65]:
# word dic
word_list = []
for elm in sentences:
    word_list += elm
word_list = list(set(word_list))
word_list.sort()
word_list = ['<pad>'] + word_list

word_dic = {word : idx for idx, word in enumerate(word_list)}

# pos dic
pos_list = []
for elm in pos:
    pos_list += elm
pos_list = list(set(pos_list))
pos_list.sort()
pos_list = ['<pad>'] + pos_list

pos_dic = {pos : idx for idx, pos in enumerate(pos_list)}

print(word_dic)
print(pos_dic)

{'<pad>': 0, 'very': 14, 'hungry': 10, 'difficult': 5, 'tensorflow': 13, 'learning': 12, 'changing': 3, 'feel': 7, 'a': 2, 'for': 8, 'framework': 9, 'deep': 4, 'is': 11, 'fast': 6, 'I': 1}
{'<pad>': 0, 'noun': 4, 'pronoun': 6, 'adjective': 1, 'determiner': 3, 'preposition': 5, 'verb': 7, 'adverb': 2}


In [66]:
sen_len, sen_indices = pad_seq(sequences = sentences, max_len = max_length, dic = word_dic)
_, pos_indices = pad_seq(sequences = pos, max_len = max_length, dic = pos_dic)

pprint(sen_len)
pprint(sen_indices)
pprint(pos_indices)

[3, 4, 7, 5]
[[1, 7, 10, 0, 0, 0, 0, 0],
 [13, 11, 14, 5, 0, 0, 0, 0],
 [13, 11, 2, 9, 8, 4, 12, 0],
 [13, 11, 14, 6, 3, 0, 0, 0]]
[[6, 7, 1, 0, 0, 0, 0, 0],
 [4, 7, 2, 1, 0, 0, 0, 0],
 [4, 7, 3, 4, 5, 1, 4, 0],
 [4, 7, 2, 1, 7, 0, 0, 0]]


## Many to Many : Simple

In [67]:
max_length = 8
h_dim = 2
n_of_classes = len(pos_dic)

seq_len = tf.placeholder(dtype = tf.int32, shape = [None])
seq_indices = tf.placeholder(dtype = tf.int32, shape = [None, max_length])
label = tf.placeholder(dtype = tf.int32, shape = [None, max_length])

one_hot = np.eye(len(word_dic)).astype(np.float32)
one_hot = tf.get_variable(name='one_hot', initializer = one_hot,
                                   trainable = False)
seq_batch = tf.nn.embedding_lookup(params = one_hot, ids = seq_indices)

In [68]:
gru_cell = tf.contrib.rnn.GRUCell(num_units = h_dim, activation = tf.nn.tanh)
score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell = gru_cell, output_size = n_of_classes)
outputs, _ = tf.nn.dynamic_rnn(cell = score_cell, inputs = seq_batch, sequence_length = seq_len,
                             dtype = tf.float32)

pprint(outputs)
pprint(_)

<tf.Tensor 'rnn/transpose_1:0' shape=(?, 8, 8) dtype=float32>
<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 2) dtype=float32>


In [69]:
masking = tf.sequence_mask(lengths = sen_len,
                           maxlen = max_length, dtype = tf.float32)
seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs,
                                                targets = label,
                                                weights = masking)

In [70]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
pprint(sess.run([outputs, _], feed_dict = {seq_len : [sen_len[0]], seq_indices : [sen_indices[0]]}))

[array([[[ 0.09354518,  0.0538936 , -0.09309661, -0.11647913, -0.00309094,
         -0.00814186, -0.08504914, -0.05891601],
        [ 0.08646086,  0.04892254, -0.10718532, -0.11372394, -0.02657486,
         -0.0292326 , -0.10519908, -0.04542852],
        [ 0.00719084,  0.00440079, -0.00102638, -0.00719475,  0.00664022,
          0.0056689 ,  0.00117315, -0.00714619],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ]]], dtype=float32

In [71]:
pprint(sess.run([masking, seq2seq_loss], feed_dict = {seq_len : sen_len, seq_indices : sen_indices,
                                           label : pos_indices}))

[array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.]], dtype=float32),
 2.0768306]
